In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
import pandas as pd
demo_url = "../input/fd-export.csv"
demo = pd.read_csv(demo_url)

In [ ]:
print (demo.head())

In [ ]:
demo.describe()

In [ ]:
# ONE MILLION RECORDS, ITS A NICE DATASET
# to put in perspective. a pharmacy collects one million validated medication records in ten years. a city like Ny collects halve a billion records per year... so to make big data extractions you need more data and consistently recorded data 



    for c in demo.columns:
        print(c,len(set(demo[c])))
    
#count per field, statistics
----
    user_id 6307
    age 43332
    sex 5
    country 96
    checkin_date
     762
    trackable_id 14640
    trackable_type 6
    trackable_name 22878
    trackable_value 3954

In [ ]:
from sklearn.preprocessing import LabelEncoder
#for c in demo.columns:
#    print(c,len(set(demo[c])))

lbl = LabelEncoder() 
lbl.fit(list(demo['user_id'].values)+list(demo['trackable_type'].values)) 
demo['user_id'] = lbl.transform(list(demo['user_id'].values))
demo['trackable_type'] = lbl.transform(list(demo['trackable_type'].values))

print(demo.head())

demo=demo.sort_values(by=['user_id','checkin_date','trackable_type'])
print(demo.head(100))
#condition=demo[demo['trackable_type']=='Condition']
#print(condition.head())
#symptom=demo[demo['trackable_type']=='Symptom']
#print(symptom.head())
#sydi=pd.merge(symptom,condition, on='user_id', how='outer')
#print(sydi)

In [ ]:
# here we have only a patient recording his diseases, nothing more, we can use it to make a combination of diseases stats, nothing else
patient=demo[demo['user_id']==46]
print(patient)

ohw serious DB consistency problems...
---
same id has different names...
same name has different spellings, 

you should clean up the database with a metathesaurus db


In [ ]:
demo['teller']=1.0

#group disnr - disease
disease_id=demo[['trackable_id','trackable_type','trackable_name','teller']].groupby(by=['trackable_id','trackable_type','trackable_name']).sum()
print(disease_id)
#find top diseases symptoms
disease_name=demo[['trackable_id','trackable_type','trackable_name','teller']].groupby(by=['trackable_name','trackable_type','trackable_id']).sum()
print(disease_name)

diseas_stat=demo[['trackable_id','trackable_name','trackable_type','teller']].groupby(by=['trackable_id','trackable_type','trackable_name']).sum()
print(diseas_stat.sort_values(by='teller'))  #14K diseas/symp numbers


In [ ]:
# if we drop the duplicate disease patient records we endup with 110K records ten times less

disperpatstat=demo[['trackable_id','trackable_type','trackable_name','teller','user_id']].drop_duplicates()
diseas_stat=disperpatstat[['trackable_id','trackable_type','trackable_name','teller']].groupby(by=['trackable_id','trackable_type','trackable_name']).sum()
print(diseas_stat.sort_values(by='teller'))

In [ ]:
#take 649 Migraine
disperpatstat=demo[['trackable_id','trackable_type','teller','user_id']].drop_duplicates()
migraine=disperpatstat[disperpatstat['trackable_id']==649]

migraine_cotrack=pd.merge(migraine, disperpatstat, on='user_id', how='inner')

#print(migraine_cotrack.pivot(index='user_id', columns='bar')['teller']
co_migr=migraine_cotrack.groupby(['trackable_id_y','trackable_type_y']).sum().sort_values('teller_y')
co_migr=co_migr[co_migr['teller_y']>1]
co_migr['freq']=co_migr['teller_y']/co_migr['teller_y'].max()
co_migr



40K trackable_id combinations
----

In [ ]:
total_cotrack=pd.merge(disperpatstat, disperpatstat, on='user_id', how='inner').groupby(['trackable_id_y','trackable_type_y']).sum()
total_cotrack['freq']=total_cotrack['teller_y']/total_cotrack['teller_y'].max()
total_cotrack


Significant problems associated with migraine
----

In [ ]:
sign_migr=pd.merge(co_migr, total_cotrack, on='user_id', how='inner')
sign_migr['anova']=sign_migr['freq_x']/sign_migr['freq_y']
sign_migr=sign_migr.drop_duplicates()
sign_migr.sort_values(by='anova')